# Qdrant

本指南将帮助您开始使用由[Qdrant向量存储](/docs/integrations/vectorstores/qdrant)支持的检索器。有关所有功能和配置的详细文档，请参阅[API参考](https://api.js.langchain.com/classes/langchain.retrievers_self_query.SelfQueryRetriever.html)。

## 概述

[自查询检索器](/docs/how_to/self_query/)通过根据输入查询动态生成元数据过滤条件来检索文档。这使得检索器在获取结果时，除了纯语义相似性之外，还可以考虑底层文档的元数据。

它使用一个称为`Translator`的模块，该模块基于关于元数据字段和特定向量存储支持的查询语言的信息生成过滤条件。

### 集成详情

| 后端向量存储 | 自托管 | 云服务 | 包 | [Python支持](https://python.langchain.com/docs/integrations/retrievers/self_query/qdrant_self_query/) |
| :--- | :--- | :---: | :---: | :---: |
[`QdrantVectorStore`](https://api.js.langchain.com/classes/langchain_qdrant.QdrantVectorStore.html) | ✅ | ✅ | [`@langchain/qdrant`](https://www.npmjs.com/package/@langchain/qdrant) | ✅ |

## 安装设置

按照[此处](/docs/integrations/vectorstores/qdrant)的文档设置Qdrant实例。设置以下环境变量：

```ts
process.env.QDRANT_URL = "YOUR_QDRANT_URL_HERE" // 例如，http://localhost:6333
```

如果您希望从各个查询中获得自动跟踪，也可以通过取消注释以下内容来设置您的[LangSmith](https://docs.smith.langchain.com/) API密钥：

```typescript
// process.env.LANGSMITH_API_KEY = "<YOUR API KEY HERE>";
// process.env.LANGSMITH_TRACING = "true";
```

### 安装

该向量存储位于`@langchain/qdrant`包中。您还需要安装`langchain`和`@langchain/community`包以导入主要的`SelfQueryRetriever`类。

在本示例中，我们还将使用OpenAI嵌入，因此您需要安装`@langchain/openai`包并[获取API密钥](https://platform.openai.com)：

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  @langchain/qdrant langchain @langchain/community @langchain/openai @langchain/core
</Npm2Yarn>
```

官方Qdrant SDK（`@qdrant/js-client-rest`）会作为`@langchain/qdrant`的依赖项自动安装，但您可能也想独立安装它。

## 实例化

首先，用一些包含元数据的文档初始化你的 Qdrant 向量存储：

In [2]:
import { OpenAIEmbeddings } from "@langchain/openai";
import { QdrantVectorStore } from "@langchain/qdrant";
import { Document } from "@langchain/core/documents";
import type { AttributeInfo } from "langchain/chains/query_constructor";

import { QdrantClient } from "@qdrant/js-client-rest";

/**
 * First, we create a bunch of documents. You can load your own documents here instead.
 * Each document has a pageContent and a metadata field. Make sure your metadata matches the AttributeInfo below.
 */
const docs = [
  new Document({
    pageContent:
      "A bunch of scientists bring back dinosaurs and mayhem breaks loose",
    metadata: { year: 1993, rating: 7.7, genre: "science fiction" },
  }),
  new Document({
    pageContent:
      "Leo DiCaprio gets lost in a dream within a dream within a dream within a ...",
    metadata: { year: 2010, director: "Christopher Nolan", rating: 8.2 },
  }),
  new Document({
    pageContent:
      "A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea",
    metadata: { year: 2006, director: "Satoshi Kon", rating: 8.6 },
  }),
  new Document({
    pageContent:
      "A bunch of normal-sized women are supremely wholesome and some men pine after them",
    metadata: { year: 2019, director: "Greta Gerwig", rating: 8.3 },
  }),
  new Document({
    pageContent: "Toys come alive and have a blast doing so",
    metadata: { year: 1995, genre: "animated" },
  }),
  new Document({
    pageContent: "Three men walk into the Zone, three men walk out of the Zone",
    metadata: {
      year: 1979,
      director: "Andrei Tarkovsky",
      genre: "science fiction",
      rating: 9.9,
    },
  }),
];

/**
 * Next, we define the attributes we want to be able to query on.
 * in this case, we want to be able to query on the genre, year, director, rating, and length of the movie.
 * We also provide a description of each attribute and the type of the attribute.
 * This is used to generate the query prompts.
 */
const attributeInfo: AttributeInfo[] = [
  {
    name: "genre",
    description: "The genre of the movie",
    type: "string or array of strings",
  },
  {
    name: "year",
    description: "The year the movie was released",
    type: "number",
  },
  {
    name: "director",
    description: "The director of the movie",
    type: "string",
  },
  {
    name: "rating",
    description: "The rating of the movie (1-10)",
    type: "number",
  },
  {
    name: "length",
    description: "The length of the movie in minutes",
    type: "number",
  },
];

/**
 * Next, we instantiate a vector store. This is where we store the embeddings of the documents.
 * We also need to provide an embeddings object. This is used to embed the documents.
 */

const client = new QdrantClient({ url: process.env.QDRANT_URL });

const embeddings = new OpenAIEmbeddings();
const vectorStore = await QdrantVectorStore.fromDocuments(docs, embeddings, {
  client,
  collectionName: "movie-collection",
});

现在我们可以实例化我们的检索器：

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />
```

In [3]:
// @lc-docs-hide-cell

import { ChatOpenAI } from "@langchain/openai";

const llm = new ChatOpenAI({
  model: "gpt-4o",
  temperature: 0,
});

In [5]:
import { SelfQueryRetriever } from "langchain/retrievers/self_query";
import { QdrantTranslator } from "@langchain/community/structured_query/qdrant";

const selfQueryRetriever = SelfQueryRetriever.fromLLM({
  llm: llm,
  vectorStore: vectorStore,
  /** A short summary of what the document contents represent. */
  documentContents: "Brief summary of a movie",
  attributeInfo: attributeInfo,
  structuredQueryTranslator: new QdrantTranslator(),
});

## 用法

现在，提出一个需要了解文档元数据才能回答的问题。你可以看到检索器将生成正确的结果：

In [6]:
await selfQueryRetriever.invoke(
  "Which movies are rated higher than 8.5?"
);

[
  Document {
    pageContent: 'A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea',
    metadata: { director: 'Satoshi Kon', rating: 8.6, year: 2006 },
    id: undefined
  },
  Document {
    pageContent: 'Three men walk into the Zone, three men walk out of the Zone',
    metadata: {
      director: 'Andrei Tarkovsky',
      genre: 'science fiction',
      rating: 9.9,
      year: 1979
    },
    id: undefined
  }
]


## 在链式应用中的使用

与其他检索器一样，Qdrant 自查询检索器可以通过[链式应用](/docs/how_to/sequence/)集成到LLM应用中。

请注意，由于其返回的答案可能在很大程度上依赖于文档的元数据，我们以不同的格式返回检索到的文档，以便包含这些信息。

In [7]:
import { ChatPromptTemplate } from "@langchain/core/prompts";
import { RunnablePassthrough, RunnableSequence } from "@langchain/core/runnables";
import { StringOutputParser } from "@langchain/core/output_parsers";

import type { Document } from "@langchain/core/documents";

const prompt = ChatPromptTemplate.fromTemplate(`
Answer the question based only on the context provided.

Context: {context}

Question: {question}`);

const formatDocs = (docs: Document[]) => {
  return docs.map((doc) => JSON.stringify(doc)).join("\n\n");
}

// See https://js.langchain.com/docs/tutorials/rag
const ragChain = RunnableSequence.from([
  {
    context: selfQueryRetriever.pipe(formatDocs),
    question: new RunnablePassthrough(),
  },
  prompt,
  llm,
  new StringOutputParser(),
]);

In [8]:
await ragChain.invoke("Which movies are rated higher than 8.5?")

The movies rated higher than 8.5 are the ones directed by Satoshi Kon (rating: 8.6) and Andrei Tarkovsky (rating: 9.9).


## 默认搜索参数

您还可以在上述方法中传入一个 `searchParams` 字段，该字段提供默认的过滤器，这些过滤器将与任何生成的查询一并应用。过滤器的语法与底层 Qdrant 向量存储相同：

In [9]:
const selfQueryRetrieverWithDefaultParams = SelfQueryRetriever.fromLLM({
  llm: llm,
  vectorStore: vectorStore,
  documentContents: "Brief summary of a movie",
  attributeInfo: attributeInfo,
  structuredQueryTranslator: new QdrantTranslator(),
  searchParams: {
    filter: {
      must: [
        {
          key: "metadata.rating",
          range: {
            gt: 8.5,
          },
        },
      ],
    },
    mergeFiltersOperator: "and",
  },
});

## API 参考文档

如需详细了解 Qdrant 自查询检索器的所有功能和配置，请访问 [API 参考文档](https://api.js.langchain.com/classes/langchain.retrievers_self_query.SelfQueryRetriever.html)。